# SVM 모델로 분류 분석 수행

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [2]:
f=pd.read_csv('final_merge.csv', encoding='euc_kr')

In [3]:
for i in range(f.shape[0]):
    if f.iloc[i,-1]=='상승':
        f.iloc[i,-1]=1
    else:
        f.iloc[i,-1]=0


In [4]:
X_train=f.iloc[0:3962,1:-1]
X_test=f.iloc[3962:,1:-1]
y_train=f.iloc[0:3962,-1]
y_test=f.iloc[3962:,-1]

In [5]:
X_train.head()

,지수종가,거래대금,상장주식수,시가총액,외국인보유주식수,외국인보유시가총액,신용거래종목수,신용자료일자,회사수,거래형성종목수,...,금괴(일간),금속,oil,미국n년,국고채권(n년),USD,EUR,CNY,JPY,GBP
0,520.95,1223328,19638887,191252891,2723222,58238292,746.0,20010102.0,704,853,...,270.6,7202.5,25.8,8.5,11.9,1259.7,1185.38,152.19,1099.07,1885.02
1,521.43,1583403,19633559,191363161,2727918,58249739,746.0,20010102.0,704,854,...,267.5,6641.3,26.5,8.6,11.8,1267.8,1205.49,153.15,1108.07,1903.35
2,558.02,2984761,19634734,204074166,2735566,62823637,746.0,20010102.0,704,855,...,267.7,6843.0,26.8,8.3,11.4,1285.4,1193.24,155.29,1128.53,1914.15
3,580.85,3569582,19637038,212089180,2747453,65820417,746.0,20010103.0,704,855,...,268.2,6878.7,26.6,8.0,11.0,1249.3,1190.90,150.92,1078.01,1876.82
4,586.65,2948092,19630735,214231273,2751599,66348238,746.0,20010104.0,704,854,...,268.3,7012.1,25.8,8.0,10.9,1267.8,1215.06,153.16,1088.71,1906.33


In [6]:
from sklearn.preprocessing import PolynomialFeatures

polynomial_svm_clf=Pipeline([
    ('poly_features', PolynomialFeatures(degree=3)),
    ('scaler', StandardScaler()),
    ('svm_clf', SVC(kernel="rbf", gamma=5, C=0.001))
])
polynomial_svm_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('poly_features',
                 PolynomialFeatures(degree=3, include_bias=True,
                                    interaction_only=False, order='C')),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=0.001, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=5,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
pred = polynomial_svm_clf.predict(X_test)
accuracy = accuracy_score(y_test , pred)
print('SVM 정확도: {0:.4f}'.format(accuracy))

SVM 정확도: 0.5524


In [8]:
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
tn, fp, fn, tp = confusion_matrix(y_true = y_test, y_pred=pred).ravel()
print("------------------------------------------------------------------")
print("True_Positive:", tp)
print("True_Negative:", tn)
print("False_Positive:", fp)
print("False_Negative:", fn)
# 리포트 확인
print("------------------------------------------------------------------")
print("[Summary]")
print(metrics.classification_report(y_test, pred))
print("Accuracy:", metrics.accuracy_score(y_true = y_test, y_pred=pred))

------------------------------------------------------------------
True_Positive: 269
True_Negative: 0
False_Positive: 218
False_Negative: 0
------------------------------------------------------------------
[Summary]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       218
           1       0.55      1.00      0.71       269

    accuracy                           0.55       487
   macro avg       0.28      0.50      0.36       487
weighted avg       0.31      0.55      0.39       487

Accuracy: 0.5523613963039015


C:\Users\chocolet\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test , pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [10]:
get_clf_eval(y_test, pred)

오차 행렬
[[  0 218]
 [  0 269]]
정확도: 0.5524, 정밀도: 0.5524, 재현율: 1.0000,    F1: 0.7116, AUC:0.5000
